# Задание

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Необходимо:

- Посчитайте, сколько книг вышло после 1 января 2000 года;

- Для каждой книги посчитайте количество обзоров и среднюю оценку;

- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Подключение к базе и импорт Pandas и SQLAlchemy

In [1]:
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432,
 'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Загружаем датасеты

## Books

In [2]:
books = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(books, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Authors

In [3]:
authors = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(authors, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


## Publishers

In [4]:
publishers = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(publishers, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Ratings

In [5]:
ratings = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(ratings, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


## Reviews

In [6]:
reviews = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(reviews, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Задания

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
books = '''
SELECT 
COUNT(title) AS cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(books, con = engine) 

,cnt
0,819


## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''
SELECT 
    books.title AS title,
    books.book_id AS id,
    AVG(ratings.rating) AS avg_rating,
    COUNT(DISTINCT reviews.text) AS cnt_reviews
FROM
    books
INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
     books.title,
     books.book_id
order by
     cnt_reviews DESC
limit 20
'''
pd.io.sql.read_sql(query, con = engine)

,title,id,avg_rating,cnt_reviews
0,Twilight (Twilight #1),948,3.662500,7
1,The Alchemist,627,3.789474,6
2,Outlander (Outlander #1),497,4.125000,6
3,The Hobbit or There and Back Again,750,4.125000,6
4,Eat Pray Love,207,3.395833,6
5,The Book Thief,656,4.264151,6
6,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
7,The Catcher in the Rye,673,3.825581,6
8,The Giver (The Giver #1),733,3.750000,6
9,The Glass Castle,734,4.206897,6


## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''select distinct publishers.publisher,  count(distinct books.book_id) as cnt_book 
           from  publishers
           inner join books on books.publisher_id = publishers.publisher_id
           where books.num_pages > 50
           group by publishers.publisher
           order by cnt_book desc'''


display(pd.io.sql.read_sql(query, con = engine).head(20))

,publisher,cnt_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,Berkley Books,14
8,St. Martin's Press,14
9,Delta,13


## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query = '''
SELECT
      SUBQ.author AS author,
      AVG(SUBQ.avg_rating) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS cnt_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''

display(pd.io.sql.read_sql(query, con = engine).head(1))

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query = '''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(text)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as cnt
FROM 
    ratings
GROUP BY
        username) AS cnt_rating
ON cnt_rating.username = reviews.username
where cnt > 50
GROUP BY reviews.username) as count;                                  
'''
display(pd.io.sql.read_sql(query, con = engine))

,round
0,24.0


# Итоги

1) После 1 января 2000 года было выпущено 819 книг.

2) Для 7 книг отсуствуют рейтинг и обзор, максимальное кол-во обзоров у книги Twilight (Twilight #1). Достаточно много книг с оценкой 5, но у них всего 2 обзора.

3) Издательство Penguin Books - выпустило наибольше число книг в кол-ве 42 книг, толще 50 страниц.

4) Cамая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré.

5) Среднее кол-во текстовых обзоров пользователей, которые поставили более чем по 50 оценок равно 24.


# Общий вывод

1) Книги с более чем 50 пользовательских оценок, можно считать правдободными.

2) Стоит обратить внимание на книги с более чем 3 обзорами, так как много завышенных оценок у книг с 2 обзорами.

3) Так же среднее количество текстовых обзоров на книги с более чем 50 оценками имеют в среднем 24 текстовых обзора, что будет достаточно информативно для будущих пользователей.